#### Parte 4: Más Casos

Como ahora tenemos más casos en nuestro poder, estos fueron integrados de la misma manera que los primeros 5 en los archivos anteriores. Las variables resultantes son las siguientes:

In [1]:
import tensorflow as tf 
import numpy as np 

wind_target  = tf.convert_to_tensor(np.load("variables/vientos_target_18_05_24.npy"))
wind_input   = tf.convert_to_tensor(np.load("variables/vientos_input_18_05_24.npy"))
lat          = tf.convert_to_tensor(np.load("variables/lat_18_05_24.npy"))
lon          = tf.convert_to_tensor(np.load("variables/lon_18_05_24.npy"))
orog         = tf.convert_to_tensor(np.load("variables/orog_18_05_24.npy"))
time2        = tf.convert_to_tensor(np.load("variables/time2_18_05_24.npy"))

wind_target  = tf.tile(wind_target, [1, 3, 1, 1, 1])
wind_input   = tf.tile(wind_input,[1, 3, 1, 1, 1]) 
lat          = tf.tile(lat, [1, 3, 1, 1, 1])
lon          = tf.tile(lon, [1, 3, 1, 1, 1])
orog         = tf.tile(orog, [1, 3, 1, 1, 1])
time2        = tf.tile(time2, [1, 3, 1, 1, 1])

casos = list( np.load("variables/casos_18_05_24.npy")) 

c:\Users\ruben\anaconda3\envs\sivar\lib\site-packages\scipy\__init__.py:169: UserWarning: A NumPy version >=1.18.5 and <1.26.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


La cantidad de casos utilizados fueron 20. Se excluyó el caso "2019_06_25" porque, de momento, cuenta con 4 días de pronóstico y no 5 como el resto. Si bien esto para la red no se traduce en un problema, sí lo es para las funciones de división de los datos que hemos construído anteriormente. 

#### División de los datos

In [2]:
%%time
from datasplit import Data_split

parches=np.zeros((wind_target.shape[0], 32, 32))

division_te= Data_split(parches=parches, seed=20)
division_te.init_etiquetas()
intervalo_temporal_te = division_te.intervalo_temporal(porc_intervalo=0.04, verbose=True)
idx_random_te         = division_te.muestra_temporal(porc_muestra=0.08, verbose=True)
tiempos_extraer_te    = division_te.extraccion_temporal(verbose=True)
idx_cuadrantes_te     = division_te.muestra_espacial(verbose=True)
idx_mantener_te       = division_te.extraccion_espacial(verbose=True)
treino_te, testeo     = division_te.extraccion_final(porc_muestra=0.05, verbose=True)

 374400 etiquetas espaciales y temporales creadas correctamente
Proceso 1: Extracción intervalo temporal
Cantidad de tiempos extraidos:  96
Cantidad de tiempos restantes:  2304
------------------------------------------------------------
Proceso 2: Extracción muestra temporal
Cantidad de tiempos extraidos:  192
Cantidad de tiempos restantes:  2112
------------------------------------------------------------
Proceso 3: Exclusión de parches dado etiquetado temporal
Cantidad de tiempos extraídos totales:  288
Cantidad de parches original:  374400
Cantidad de parches extraídos:  44928
Cantidad de parches restantes:  329472
------------------------------------------------------------
Proceso 4: Extracción de cuadrantes
Cantidad de cuadrantes extraidos:  7
Cantidad de cuadrantes restantes:  149
------------------------------------------------------------
Proceso 5: Exclusión de parches dado etiquetado cuadrante.
Cantidad de parches extraídos:  16800
Cantidad de parches posterior extracción e

In [3]:
division_va=Data_split(parches=parches,indices=treino_te, seed=20)
division_va.init_etiquetas()
intervalo_temporal_va = division_va.intervalo_temporal(porc_intervalo=0.04, indices= tiempos_extraer_te, verbose=True )
idx_random_va         = division_va.muestra_temporal(porc_muestra=0.08, verbose=True)
tiempos_extraer_va    = division_va.extraccion_temporal(verbose=True)
idx_cuadrantes_va     = division_va.muestra_espacial(indices=idx_cuadrantes_te, verbose=True)
idx_mantener_va       = division_va.extraccion_espacial(verbose=True)
treino, validacion = division_va.extraccion_final( porc_muestra=0.08, verbose=True)

 298953 etiquetas espaciales y temporales creadas correctamente
Proceso 1: Extracción intervalo temporal
Cantidad de tiempos extraidos:  96
Cantidad de tiempos restantes:  2016
------------------------------------------------------------
Proceso 2: Extracción muestra temporal
Cantidad de tiempos extraidos:  192
Cantidad de tiempos restantes:  1824
------------------------------------------------------------
Proceso 3: Exclusión de parches dado etiquetado temporal
Cantidad de tiempos extraídos totales:  288
Cantidad de parches original:  298953
Cantidad de parches extraídos:  40706
Cantidad de parches restantes:  258247
------------------------------------------------------------
Proceso 4: Extracción de cuadrantes
Cantidad de cuadrantes extraidos:  7
Cantidad de cuadrantes restantes:  142
------------------------------------------------------------
Proceso 5: Exclusión de parches dado etiquetado cuadrante.
Cantidad de parches extraídos:  14060
Cantidad de parches posterior extracción e

##### Chequeo independencia de índices de los conjuntos

In [4]:
set_entrenamiento= set(treino)
set_testeo       = set(testeo)
set_validacion   = set(validacion)

suma_conjuntos= len(treino) + len(validacion) + len(testeo)

if suma_conjuntos == parches.shape[0]:
    print("No existen elementos repetidos")
else: 
    print("Los conjuntos no están bien separados")

No existen elementos repetidos


#### Filtracion de los parches:

In [5]:
def extraccion_parches( var, treino=treino, testeo=testeo, validacion=validacion):

    tr = tf.gather(var, treino)
    va = tf.gather(var, validacion)
    te = tf.gather(var, testeo)

    return tr, va, te


target_tr, target_va, target_te = extraccion_parches( wind_target)
input_tr , input_va, input_te   = extraccion_parches( wind_input)
orog_tr,   orog_va, orog_te     = extraccion_parches( orog)
lat_tr, lat_va, lat_te          = extraccion_parches( lat)
lon_tr, lon_va, lon_te          = extraccion_parches( lon)
time_tr, time_va, time_te       = extraccion_parches( time2) 

print(f"Shape target_tr {target_tr.shape}, target_va: {target_va.shape}, target_te: {target_te.shape}" )

Shape target_tr (226418, 3, 32, 32, 1), target_va: (72535, 3, 32, 32, 1), target_te: (75447, 3, 32, 32, 1)


##### Vaciado de recursos:

In [6]:
wind_target  = None
wind_input   = None
lat          = None
lon          = None
orog         = None
time2        = None

#### Normalización de los parches

Normalizaremos CADA VARIABLE del conjunto de entrenamiento en el rango $[0,1]$ utilizando la siguiente fórmula:
$$ \text{normalización(variable)} := \dfrac{variable - \min(variable_{tr})}{\max(variable_{tr}) - \min(variable_{tr})}= \text{variable norm}$$
donde $\min(variable_{tr})$ y $\max(variable_{tr})$ corresponderán al mínimo y máximo valor de nuestra variable del conjunto de entrenamiento a normalizar respectivamente y serán guardados como factores de escalado para el resto de variables y para la posterior desnormalización.

In [7]:
def normalizacion( data_in,factores_escala=None,inversa=False):

    '''Método que normaliza los valores entregados.
    input:
        data_in: [tensor] Tensor con los parches de la variable que forma parte del input de la Red
        inverse: [Boolean, default=False] True en caso de devolver la transformación
    return: 
        variable normalizada
    '''

    if not factores_escala:              
            factor_min  = tf.reduce_min(data_in)
            factor_max  = tf.reduce_max(data_in)

            factores_escala=[factor_min, factor_max ]
    
    if not inversa:
        data_out= (data_in - factores_escala[0]) / (factores_escala[1] - factores_escala[0])

    else:
        data_out=(data_in * (factores_escala[1]- factores_escala[0]))+factores_escala[0]
    
    return data_out, factores_escala

In [8]:
target_tr_norm, factores_target_tr = normalizacion(target_tr)
target_va_norm, _ = normalizacion(target_va, factores_escala=factores_target_tr)
target_te_norm, _ = normalizacion(target_te, factores_escala=factores_target_tr)

input_tr_norm, factores_input_tr  = normalizacion(input_tr)
input_va_norm, _ = normalizacion(input_va, factores_escala=factores_input_tr)
input_te_norm, _ = normalizacion(input_te, factores_escala=factores_input_tr)

orog_tr_norm,  factores_orog_tr   = normalizacion(orog_tr)
orog_va_norm, _ = normalizacion(orog_va, factores_escala=factores_orog_tr)
orog_te_norm, _ = normalizacion(orog_te, factores_escala=factores_orog_tr)

lat_tr_norm, factores_lat_tr      = normalizacion(lat_tr)
lat_va_norm, _ = normalizacion(lat_va, factores_escala=factores_lat_tr)
lat_te_norm, _ = normalizacion(lat_te, factores_escala=factores_lat_tr)

lon_tr_norm, factores_lon_tr      = normalizacion(lon_tr)
lon_va_norm, _ = normalizacion(lon_va, factores_escala=factores_lon_tr)
lon_te_norm, _ = normalizacion(lon_te, factores_escala=factores_lon_tr)

time_tr_norm, factores_time_tr    = normalizacion(time_tr)
time_va_norm, _ = normalizacion(time_va, factores_escala=factores_time_tr)
time_te_norm, _ = normalizacion(time_te, factores_escala=factores_time_tr)

#### Comprobación desnormalización: 

Debido al error de punto flotante que se arrasta al realizar operaciones, es esperable que el proceso de desnormalización no entregue los valores originales de la variable tratada:

In [9]:
target_tr_desnorm,_ = normalizacion(target_tr_norm, factores_escala= factores_target_tr, inversa= True)
target_va_desnorm,_ = normalizacion(target_va_norm, factores_escala= factores_target_tr, inversa= True)
target_te_desnorm,_ = normalizacion(target_te_norm, factores_escala= factores_target_tr, inversa= True)


# Comparación de los tensores:
comp_treino     = tf.reduce_all(tf.equal(target_tr, target_tr_desnorm))
comp_validacion = tf.reduce_all(tf.equal(target_va, target_va_desnorm))
comp_testeo     = tf.reduce_all(tf.equal(target_te, target_te_desnorm))

print(f"Comparaciones: {comp_treino} - {comp_validacion} - {comp_testeo}")

Comparaciones: False - False - False


Sin embargo, esta diferencia no es significativa ni tampoco se presenta en todos los valores:

In [10]:
import tensorflow as tf

tol = 0.000002
dif_tr = tf.abs(target_tr - target_tr_desnorm)
dif_va = tf.abs(target_va - target_va_desnorm)
dif_te = tf.abs(target_te - target_te_desnorm)

diferencias = [dif_tr, dif_va, dif_te]

for dif in diferencias: 

    comparacion = tf.less_equal(dif, tol)
    resultado = tf.reduce_all(comparacion)

    if resultado: 
        print("La diferencia es menor a la tolerancia")

La diferencia es menor a la tolerancia
La diferencia es menor a la tolerancia
La diferencia es menor a la tolerancia


#### Últimos retoques
##### Variable time2:

In [11]:
time_tr_norm=time_tr_norm[:,:,:4,:4,:]
time_va_norm=time_va_norm[:,:,:4,:4,:]
time_te_norm=time_te_norm[:,:,:4,:4,:]
print("Dimensiones time entrenamiento: ", time_tr_norm.shape)

Dimensiones time entrenamiento:  (226418, 3, 4, 4, 1)


Visualización del resto de dimensiones:

In [12]:
print("Dimensiones variables entrenamiento: ",input_tr_norm.shape)
print("Dimensiones variables validación: ",input_va_norm.shape)
print("Dimensiones variables testeo: ",input_te_norm.shape)

Dimensiones variables entrenamiento:  (226418, 3, 32, 32, 1)
Dimensiones variables validación:  (72535, 3, 32, 32, 1)
Dimensiones variables testeo:  (75447, 3, 32, 32, 1)


In [13]:
factores_target_tr

[<tf.Tensor: shape=(), dtype=float32, numpy=0.00012237605>,
 <tf.Tensor: shape=(), dtype=float32, numpy=45.299717>]

In [14]:
#### Guardado de parches: 
import os 

def guardado_parches(parches, nombre_parches, carpeta, opcional='', verbose=False):
    ruta_completa = os.path.join(carpeta, nombre_parches + opcional + '.npy')
    np.save(ruta_completa, parches)
    if verbose:
        print("Parches guardados exitosamente como: ", ruta_completa)


guardado_parches(factores_target_tr, "inp_factor", "parches", "_180524_20")
guardado_parches(factores_lat_tr, "lat_factor", "parches", "_180524_20")
guardado_parches(factores_lon_tr, "lon_factor", "parches", "_180524_20")
guardado_parches(factores_orog_tr, "orog_factor", "parches", "_180524_20")
guardado_parches(factores_time_tr, "time2_factor", "parches", "_180524_20")
guardado_parches(factores_target_tr, "target_factor", "parches", "_180524_20")

guardado_parches(input_tr_norm, "inp_tr_norm", "parches", "_180524_20")
guardado_parches(input_va_norm, "inp_va_norm", "parches", "_180524_20")
guardado_parches(input_te_norm, "inp_te_norm", "parches", "_180524_20")

guardado_parches(target_tr_norm, "target_tr_norm", "parches", "_180524_20")
guardado_parches(target_va_norm, "target_va_norm", "parches", "_180524_20")
guardado_parches(target_te_norm, "target_te_norm", "parches", "_180524_20")

guardado_parches(orog_tr_norm, "orog_tr_norm", "parches", "_180524_20")
guardado_parches(orog_va_norm, "orog_va_norm", "parches", "_180524_20")
guardado_parches(orog_te_norm, "orog_te_norm", "parches", "_180524_20")

guardado_parches(lon_tr_norm, "lon_tr_norm", "parches", "_180524_20")
guardado_parches(lon_va_norm, "lon_va_norm", "parches", "_180524_20")
guardado_parches(lon_te_norm, "lon_te_norm", "parches", "_180524_20")

guardado_parches(lat_tr_norm, "lat_tr_norm", "parches", "_180524_20")
guardado_parches(lat_va_norm, "lat_va_norm", "parches", "_180524_20")
guardado_parches(lat_te_norm, "lat_te_norm", "parches", "_180524_20")

guardado_parches(time_tr_norm, "time_tr_norm", "parches", "_180524_20")
guardado_parches(time_va_norm, "time_va_norm", "parches", "_180524_20")
guardado_parches(time_te_norm, "time_te_norm", "parches", "_180524_20", verbose=True)

Parches guardados exitosamente como:  parches\time_te_norm_180524_20.npy


In [19]:
idx_dic={"treino:": treino,
         "validacion":validacion,
        "testeo": testeo}

In [21]:
import pickle
with open('indices_datos.pickle', 'wb') as archivo:
    pickle.dump(idx_dic, archivo)

print("Diccionario guardado localmente como 'mi_diccionario.pickle'")


Diccionario guardado localmente como 'mi_diccionario.pickle'


In [22]:
import pickle

# Cargar el diccionario desde el archivo
with open('indices_datos.pickle', 'rb') as archivo:
    diccionario_cargado = pickle.load(archivo)

print("Diccionario cargado desde 'mi_diccionario.pickle':", diccionario_cargado)


Diccionario cargado desde 'mi_diccionario.pickle': {'treino:': [167790, 327093, 36363, 364400, 320546, 128823, 80332, 152700, 245947, 129660, 55227, 265877, 283720, 37644, 1952, 121115, 23296, 354246, 161535, 155962, 287167, 123643, 164392, 323389, 361888, 187686, 314198, 37856, 24767, 229092, 190052, 82092, 367458, 3981, 275312, 285956, 212113, 25784, 368875, 293162, 218592, 245816, 144757, 280142, 6278, 68867, 121538, 127757, 240844, 326760, 38010, 905, 15969, 82094, 292729, 80313, 244213, 168409, 270341, 237956, 29237, 231116, 327227, 151245, 306551, 123929, 181418, 19218, 314160, 257674, 52587, 355021, 245907, 156791, 361074, 330869, 223706, 164967, 163831, 113261, 1584, 311145, 303149, 370352, 186594, 34899, 231016, 351429, 198635, 137096, 89413, 301323, 77884, 127434, 183936, 357319, 143184, 158388, 80194, 246095, 20137, 147458, 51423, 34452, 292395, 317472, 153885, 8176, 296489, 45372, 192840, 93035, 216533, 54414, 128652, 301821, 285020, 2817, 53670, 267527, 152374, 147984, 212